In [27]:
import pandas as pd
import numpy as np
import glob
import os
import datetime

In [28]:
%cd /Volumes/SEB_USB/Season_10/

/Volumes/SEB_USB/Season_10


In [29]:
## s10_flir csv file is the same as the combined RGB_flir, but without the RGB data
thermal = pd.read_csv('Haiquan/s10_flir.csv')
RGB = pd.read_csv('Haiquan/stereoTop_full_season_clustering.csv')

Rgb_flir = pd.read_csv('Haiquan/s10_flir_rgb_clustering_v4.csv')

In [30]:
del thermal['treatment']

thermal['date'] = pd.to_datetime(thermal['date'])
RGB['date'] = pd.to_datetime(RGB['date'])

thermal = thermal.sort_values(by = ['date', 'plant_name'])
RGB = RGB.sort_values(by = ['date', 'plant_name'])

In [31]:
# ## Assessing the amount of NaN values in columns
# percent_missing = Rgb_flir.isnull().sum() * 100 / len(Rgb_flir)
# missing_value_df = pd.DataFrame({'column_name': Rgb_flir.columns,
#                                  'percent_missing': percent_missing})

# missing_value_by_date_df = Rgb_flir.groupby('date')['plant_name'].apply(lambda x: x.isnull().sum()/len(x)*100)

In [32]:
# nan_df = Rgb_flir[Rgb_flir['plant_name'].isna()]

In [33]:
thermal = thermal.reset_index()
del thermal['index']

RGB = RGB.reset_index()
del RGB['index']

In [34]:
RGB = RGB.set_index(['date', 'plant_name'])
thermal = thermal.set_index(['date', 'plant_name'])

In [35]:
rgb_flir_merged = RGB.join(thermal).reset_index()

## Concatenating all of the ps2 csv files and then importing them

In [38]:
os.chdir("Haiquan/PS2/")

df = pd.DataFrame()
for file in glob.glob("*.csv", recursive = True):
     if file.endswith('.csv'):
            csv = pd.read_csv(file)
            csv['filename'] = os.path.basename(file)
            df = df.append(csv)
            del df['Unnamed: 0']

In [39]:
## Organize the ps2 file
df['Plot'] = df['Plot'].str.replace(' ','_')
df['date'] = df['filename'].str[0:10]

df = df.sort_values(by = ['date', 'Plot'])

In [40]:
df = df.reset_index()
del df['index']
df = df.rename(columns = {'Plot':'plot'})
del df['filename']

In [41]:
df['date'] = pd.to_datetime(df['date']).dt.date
rgb_flir_merged['date'] = pd.to_datetime(rgb_flir_merged['date']).dt.date

In [46]:
df = df.sort_values(by = ['date', 'plot'])
rgb_flir_merged = rgb_flir_merged.sort_values(by = ['date', 'plot'])

## Attempting to only use plot number rather than the entire string

In [50]:
rgb_flir_merged['Range'] = rgb_flir_merged['plot'].str.split('_').str[6]
df['Range'] = df['plot'].str.split('_').str[6]

rgb_flir_merged['Column'] = rgb_flir_merged['plot'].str.split('_').str[8]
df['Column'] = df['plot'].str.split('_').str[8]

In [51]:
df['Range'] = df['Range'].str.zfill(2)
rgb_flir_merged['Range'] = rgb_flir_merged['Range'].str.zfill(2)

df['Column'] = df['Column'].str.zfill(2)
rgb_flir_merged['Column'] = rgb_flir_merged['Column'].str.zfill(2)

In [52]:
df['Plot_num'] = df['Range'] + df['Column']
rgb_flir_merged['Plot_num'] = rgb_flir_merged['Range'] + rgb_flir_merged['Column']

In [54]:
del df['Range']
del df['Column']
del rgb_flir_merged['Range']
del rgb_flir_merged['Column']


In [58]:
del df['plot']

## Prepping for Merge

In [59]:
df = df.set_index(['date', 'Plot_num'])
rgb_flir_merged = rgb_flir_merged.set_index(['date', 'Plot_num'])

In [62]:
merged = rgb_flir_merged.join(df).reset_index()

In [66]:
# merged.to_csv('s10_Haiquan_updated.csv')